In [4]:
# !pip install -r requirement.txt

# !pip install sentence_transformers

# !pip install sagemaker

# Automatic Prompt Engineering (APE)

## copy of kendra_chat_llm.py code

In [157]:
##-------------------------------------##
## modified from ../kendra_chat_llm.py
##-------------------------------------##

from langchain.retrievers import AmazonKendraRetriever
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain import SagemakerEndpoint
from langchain.llms.sagemaker_endpoint import LLMContentHandler
from sentence_transformers import SentenceTransformer, util

from tqdm import tqdm
import pandas as pd
import json
import sys
import os

class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'


MAX_HISTORY_LENGTH = 5

model_parameters = {           
    "max_new_tokens": 200, 
    "temperature":0.1, 
    "seed":0, 
    "stop": ["Human:"], 
    "num_beams":1, 
    "return_full_text": False
    }


class ContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs: dict) -> bytes:
        input_str = json.dumps({"inputs": prompt, "parameters": {**model_kwargs}})
        return input_str.encode('utf-8')

    def transform_output(self, output: bytes) -> str:
        response_json = json.loads(output.read().decode("utf-8"))
        return response_json[0]["generated_text"]

    
def build_chain(prompt_template=None):
    region = os.environ["AWS_REGION"]
    kendra_index_id = os.environ["KENDRA_INDEX_ID"]
    endpoint_name = os.environ["SAGEMAKER_LLM_ENDPOINT"]


    content_handler = ContentHandler()

    llm = SagemakerEndpoint(
        endpoint_name=endpoint_name,
        region_name=region,
        model_kwargs=model_parameters,
        content_handler=content_handler
    )

    retriever = AmazonKendraRetriever(index_id=kendra_index_id)


    PROMPT = PromptTemplate(
        template=prompt_template, input_variables=["context", "question"]
    )

    condense_qa_template = """
        Given the following conversation and a follow up question, rephrase the follow up question 
        to be a standalone question.

        Chat History:
        {chat_history}
        Follow Up Input: {question}
        Standalone question:"""
    
    standalone_question_prompt = PromptTemplate.from_template(
        condense_qa_template)

    qa = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=retriever,
        condense_question_prompt=standalone_question_prompt,
        return_source_documents=False,
        combine_docs_chain_kwargs={"prompt": PROMPT})
    return qa


def run_chain(chain, prompt: str, history=[]):
    result = chain({"question": prompt, "chat_history": history})
    return result

### configure AWS_REGION, KENDRA_INDEX_ID, SAGEMAKE_LLM_ENDPOINT variables

In [158]:
import os
os.environ['AWS_REGION'] = 'us-east-1'
os.environ["KENDRA_INDEX_ID"] = '6bf94090-16d3-4c34-b541-2eaadb4fe5f1'
os.environ["SAGEMAKER_LLM_ENDPOINT"] = "huggingface-pytorch-tgi-inference-2023-07-18-11-41-50-057"

### sample of Q & A test bank

In [159]:
def get_test_bank():
    test_bank = """Question: What is AWS Kendra?
    Answer: AWS Kendra is an intelligent search service powered by machine learning that provides natural language search capabilities for various data sources and content.

    Question: What types of data sources does AWS Kendra support for indexing?
    Answer: AWS Kendra supports various data sources, including Amazon S3, SharePoint Online, Salesforce, ServiceNow, Relational Databases, and more.

    Question: How does AWS Kendra handle natural language queries?
    Answer: AWS Kendra uses machine learning models to understand natural language queries and provide relevant results using contextual understanding and ranking.

    Question: What are the benefits of using AWS Kendra for enterprise search?
    Answer: Some benefits of AWS Kendra include improved search accuracy, reduced time to find relevant information, and support for multiple data sources.

    Question: How does AWS Kendra ensure security and compliance?
    Answer: AWS Kendra encrypts data at rest and in transit, provides access control via AWS Identity and Access Management (IAM), and is compliant with various security standards.

    Question: Can AWS Kendra be used for both internal and external search scenarios?
    Answer: Yes, AWS Kendra can be used for both internal enterprise search (intranet) and external customer-facing search (internet) scenarios.

    Question: What are custom data sources in AWS Kendra?
    Answer: Custom data sources in AWS Kendra allow you to integrate your own data repositories or applications into the search index.

    Question: Can AWS Kendra search across multiple languages?
    Answer: Yes, AWS Kendra supports multiple languages and can perform multilingual search queries.

    Question: How does AWS Kendra handle synonyms and acronyms?
    Answer: AWS Kendra automatically recognizes synonyms and acronyms, improving search accuracy and understanding user queries.

    Question: Does AWS Kendra support natural language processing (NLP)?
    Answer: Yes, AWS Kendra uses natural language processing to understand complex user queries and return relevant search results."""

    tmp = [i for i in test_bank.split("\n") if i != '']
    qa_dict = {}

    for qq, aa in [tmp[i: i + 2] for i in range(0, len(tmp), 2)]:
        qa_dict[qq.split('Question: ')[-1]] = aa.split('Answer: ')[-1]
    return qa_dict

### define evaluation criteria, cosine similarity between the ground truth and the generated answer

In [160]:
# Load a pre-trained SentenceTransformer model
model_name = "paraphrase-mpnet-base-v2"
model = SentenceTransformer(model_name)

# evaluation score
def get_ans_score(ground_truth, generated_answer, model):
    generated_answer_embedding = model.encode(generated_answer, convert_to_tensor=True)
    ground_truth_embedding = model.encode(ground_truth, convert_to_tensor=True)
    return util.pytorch_cos_sim(generated_answer_embedding, ground_truth_embedding)

### define prompt template used in RAG + Kendra, and set up the Q & A.

In [163]:
prompt_template = """
    The following is a friendly conversation between a human and an AI. 
    The AI is talkative and provides lots of specific details from its context.
    If the AI does not know the answer to a question, it truthfully says it 
    does not know.
    {context}
    Instruction: Based on the above documents, provide a detailed answer for, {question} Answer "don't know" 
    if not present in the document. 
    Solution:"""

qa_bank = get_test_bank()
chain = build_chain(prompt_template=prompt_template)

### get initial score with j

In [ ]:
score_dict = {}
for qq, aa in tqdm(qa_bank.items()):
    ans = run_chain(chain, prompt = qq)
    score = get_ans_score(aa, ans['answer'].strip(), model=model)
    score_dict[qq] = {'generated_answer': ans['answer'].strip(), 'correct_answer': aa, 'score': score.item()}

In [ ]:
pd.DataFrame(score_dict)

***

In [147]:
region = os.environ["AWS_REGION"]
kendra_index_id = os.environ["KENDRA_INDEX_ID"]
endpoint_name = os.environ["SAGEMAKER_LLM_ENDPOINT"]

content_handler = ContentHandler()
llm = SagemakerEndpoint(
    endpoint_name=endpoint_name,
    region_name=region,
    model_kwargs=model_parameters,
    content_handler=content_handler
)

prompt_template_improve = """Suggest a more detailed prompt for the following enclosed in three single quotes. 
                            The prompt should still be an question. The prompt should be different with the original. 
                            '''{input_text}''' """
prompt_improve = PromptTemplate.from_template(prompt_template_improve)

In [148]:
score_dict_updated = {}
for qq, aa in tqdm(qa_bank.items()):
    qq_refined = llm(prompt_improve.format(input_text=qq)).strip()
    print(qq, qq_refined)
    ans = run_chain(chain, prompt = qq_refined)
    score = get_ans_score(aa, ans['answer'].strip(), model=model)
    score_dict_updated[qq] = {'generated_answer': ans['answer'].strip(), 'correct_answer': aa, 'score': score.item()}

  0%|          | 0/10 [00:00<?, ?it/s]

What is AWS Kendra? What is AWS Kendra?


 10%|█         | 1/10 [00:07<01:07,  7.46s/it]

What types of data sources does AWS Kendra support for indexing? What types of data sources does AWS Kendra support for indexing?


 20%|██        | 2/10 [00:15<01:02,  7.78s/it]

...

 80%|████████  | 8/10 [01:03<00:16,  8.01s/it]

How does AWS Kendra handle synonyms and acronyms? What are some common synonyms and acronyms used in AWS Kendra and how does the platform handle them?


 90%|█████████ | 9/10 [01:11<00:08,  8.09s/it]

Does AWS Kendra support natural language processing (NLP)? What specific natural language processing (NLP) capabilities does AWS Kendra offer?


100%|██████████| 10/10 [01:20<00:00,  8.02s/it]


In [146]:
pd.DataFrame(score_dict_updated)

,What is AWS Kendra?,What types of data sources does AWS Kendra support for indexing?,How does AWS Kendra handle natural language queries?,What are the benefits of using AWS Kendra for enterprise search?,How does AWS Kendra ensure security and compliance?,Can AWS Kendra be used for both internal and external search scenarios?,What are custom data sources in AWS Kendra?,Can AWS Kendra search across multiple languages?,How does AWS Kendra handle synonyms and acronyms?,Does AWS Kendra support natural language processing (NLP)?
generated_answer,AWS Kendra is a cloud-based service that provi...,AWS Kendra supports indexing documents in a va...,AWS Kendra is a natural language processing (N...,The main advantages of using AWS Kendra for en...,AWS Kendra takes several measures to ensure se...,AWS Kendra can be used to improve search resul...,A custom data source is a data source that is ...,AWS Kendra can search across multiple language...,AWS Kendra handles synonyms and acronyms by us...,1. Understand the structure of the natural lan...
correct_answer,AWS Kendra is an intelligent search service po...,"AWS Kendra supports various data sources, incl...",AWS Kendra uses machine learning models to und...,Some benefits of AWS Kendra include improved s...,AWS Kendra encrypts data at rest and in transi...,"Yes, AWS Kendra can be used for both internal ...",Custom data sources in AWS Kendra allow you to...,"Yes, AWS Kendra supports multiple languages an...",AWS Kendra automatically recognizes synonyms a...,"Yes, AWS Kendra uses natural language processi..."
score,0.767183,0.741995,0.670145,0.738057,0.746491,0.782671,0.627521,0.754128,0.596884,0.273528


***